In [1]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
%watermark -v -p numpy,sklearn,pandas
%watermark -v -p cv2,PIL,matplotlib
%watermark -v -p torch,torchvision,torchaudio,pytorch_lightning
%matplotlib inline
%config InlineBackend.figure_format='retina'
%config IPCompleter.use_jedi = False

from IPython.display import display, HTML, Javascript
display(HTML('<style>.container { width:%d%% !important; }</style>' % 90))

def _IMPORT_(x):
    try:
        exec(x, globals())
    except:
        pass


CPython 3.6.9
IPython 7.16.1

numpy 1.18.5
sklearn 0.24.0
pandas 1.1.5
CPython 3.6.9
IPython 7.16.1

cv2 4.5.1
PIL 6.2.2
matplotlib 3.3.3
CPython 3.6.9
IPython 7.16.1

torch 1.8.0.dev20210103+cu101
torchvision 0.9.0.dev20210103+cu101
torchaudio not installed
pytorch_lightning 1.2.0


In [2]:
###
### Common ###
###

import os, io, time, random, math, base64
import json, requests, sys

_IMPORT_('import numpy as np')
_IMPORT_('import pandas as pd')
_IMPORT_('from tqdm.notebook import tqdm')

###
### Torch ###
###

_IMPORT_('import torch')
_IMPORT_('import torch.nn as nn')
_IMPORT_('import torch.nn.functional as F')
_IMPORT_('import torch.optim as O')
_IMPORT_('from torchvision import models as M')
_IMPORT_('from torchvision import transforms as T')
_IMPORT_('from torch.utils.data import Dataset, DataLoader')

###
### Display ###
###

_IMPORT_('import cv2')
_IMPORT_('from PIL import Image')
_IMPORT_('from torchvision.utils import make_grid')
_IMPORT_('import matplotlib.pyplot as plt')
_IMPORT_('import plotly')
_IMPORT_('import plotly.graph_objects as go')

# plotly.offline.init_notebook_mode(connected=False)

def show_video(video_path, width=None, height=None):
    W, H = '', ''
    if width:
        W = 'width=%d' % width
    if height:
        H = 'height=%d' % height
    mp4 = open(video_path, 'rb').read()
    data_url = 'data:video/mp4;base64,' + base64.b64encode(mp4).decode()
    return HTML('<video %s %s controls src="%s" type="video/mp4"/>' % (W, H, data_url))

def show_image(image_path, width=None, height=None):
    W, H = '', ''
    if width:
        W = 'width=%d' % width
    if height:
        H = 'height=%d' % height
    img = open(image_path, 'rb').read()
    data_url = 'data:image/jpg;base64,' + base64.b64encode(img).decode()
    return HTML('<img %s %s src="%s"/>' % (W, H, data_url))

from k12libs.utils.nb_easy import RACEURL

In [3]:
API_INFERENCE = f'{RACEURL}/raceai/framework/inference'
API_POPMSG = f'{RACEURL}/raceai/private/popmsg'
MODEL_TASK = 'zmq.alphapose.inference'
MSGKEY = 'zmq.alphapose.test'
TEST_VIDEO = 'https://raceai.s3.didiyunapi.com/data/media/videos/repnet_test.mp4'
API_INFERENCE, API_POPMSG 

('http://116.85.5.40:9119/raceai/framework/inference',
 'http://116.85.5.40:9119/raceai/private/popmsg')

## Halpe26 Dataset

ID | Name
:---:|:---:
<img width=200/>|<img width=300/>
0 |Nose
1 |LEye
2 |REye
3 |LEar
4 |REar
5 |LShoulder
6 |RShoulder
7 |LElbow
8 |RElbow
9 |LWrist
10|RWrist
11|LHip
12|RHip
13|LKnee
14|Rknee
15|LAnkle
16|RAnkle
17 |Head
18 |Neck
19 |Hip
20|LBigToe
21|RBigToe
22|LSmallToe
23|RSmallToe
24|LHeel
25|RHeel

## 接口测试

### 输入

```json
{
    "task": "zmq.alphapose.inference",  // zmq的任务类型, 不用修改
    "cfg": {
        "pigeon": { // 信鸽数据, key的内容由后端自行添加, 其中必须含有msgkey和user_code, 可以任意添加其他key, 返回时会携带pigeon
            "msgkey": "zmq.alphapose.test",  // 任务过程产生的数据, 会发送到msgkey的redis队列中
            "user_code": "123", // 这个任务属于哪个用户
        },
        "report_kps": False, // 实时上报帧的关键点(上面描述的形式), 建议不开启, 数据量较大
        "video": "https://raceai.s3.didiyunapi.com/data/media/videos/alphapose_test.mp4", // oss平台的视频地址
        "qsize": 1024,       // 不用修改, 开发调试用
        "save_video": True,  // 标有关键点的视频
        "save_img": False,   // 建议不要开启, 帧数太多
        "vis_fast": True     // 建议开启
    }
}
```

In [4]:
cfgdata = '''{
    "task": "%s",
    "cfg": {
        "pigeon": {
            "msgkey": "%s",
            "user_code": "123",
        },
        "report_kps": False,
        "video": "%s",
        "qsize": 1024,
        "save_video": False,
        "save_img": False,
        "vis_fast": True
    }
}''' % (MODEL_TASK, MSGKEY, TEST_VIDEO)

print('Input:\n\n', cfgdata)

Input:

 {
    "task": "zmq.alphapose.inference",
    "cfg": {
        "pigeon": {
            "msgkey": "zmq.alphapose.test",
            "user_code": "123",
        },
        "report_kps": False,
        "video": "https://raceai.s3.didiyunapi.com/data/media/videos/repnet_test.mp4",
        "qsize": 1024,
        "save_video": False,
        "save_img": False,
        "vis_fast": True
    }
}


### 输出

```
{
    "pigeon": {  // 信鸽数据, 直接透传回去
        "msgkey": "zmq.alphapose.test",
        "user_code": "123"
    },
    "task": "zmq.alphapose.inference",
    "errno": 0,
    "progress": 100.0,  // 任务的进度
    "target_json": "https://raceai.s3.didiyunapi.com/outputs/123/alphapose-results.json", // 只有进度为100.0时, 返回关键点结果json数据
    "target_mp4": "https://raceai.s3.didiyunapi.com/outputs/123/alphapose-target.mp4" // 只有进度为100.0时, 返回带有关键点标注的mp4数据
}
```

In [9]:
reqdata = eval(cfgdata)
json.loads(requests.get(url=f'{API_POPMSG}?key={MSGKEY}').text)
json.loads(requests.post(url=API_INFERENCE, json=reqdata).text)

{'errno': 0}

In [8]:
for _ in range(200):
    result = json.loads(requests.get(url=f'{API_POPMSG}?key={MSGKEY}').text)
    if len(result) > 0 and 'progress' in result[-1]:
        progress = int(result[-1]['progress'])
        print("\r", end="")
        print("Progress: {}%: ".format(progress), "▋" * (progress // 2), end="")
        sys.stdout.flush()
        if progress >= 100.0:
            print('\nOutput:\n\n', json.dumps(result[-1], indent=4))
            break
    time.sleep(3)

Progress: 100%:  ▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋
Output:

 {
    "pigeon": {
        "msgkey": "zmq.alphapose.test",
        "user_code": "123"
    },
    "task": "zmq.alphapose.inference",
    "errno": 0,
    "progress": 100.0,
    "target_json": "https://raceai.s3.didiyunapi.com/outputs/123/alphapose-results.json"
}


## 上报的关键点数据

```json
[
  {
    "image_id": "0.jpg",         // video frame image name
    "category_id": 1,            // 1 for person
    "score": 2.8476829528808594, // detect confidence score
    "idx": [
      0
    ],
    "box": [                     // detect box
      837.0687255859375,         // box: x axis
      200.54270935058594,        // box: y axis
      25.557861328125,           // box: width
      62.62861633300781          // box: height
    ],
    "keypoints": [
      850.1534423828125,    // kp0: x axis
      207.0869140625,       // kp0: y axis
      0.851284384727478,    // kp0: confidence score
      850.7650756835938,    // kp1: x axis
      205.8636932373047,    // kp2: y axis
      0.887949526309967,
      848.9302368164062,

      ...
        
      854.4346923828125,
      261.52001953125,
      0.8205215334892273,
      834.86328125,          // kp25: x aixs    
      260.2967834472656,     // kp25: y aixs
      0.9024990797042847     // kp25: confidence score
    ]
  },
 ...
]
```

## References

1.[数据集Halpe26](https://github.com/Fang-Haoshu/Halpe-FullBody)